## **서울시 작은도서관 데이터 분석**

### **데이터 불러오기**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

In [31]:
# 엑셀 불러오기
data = pd.read_excel('/content/drive/MyDrive/Contest/Library/Dataset/도서관_현황정보.xlsx')

In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 954 entries, 0 to 953
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   도서관명                   954 non-null    object 
 1   분관여부                   954 non-null    object 
 2   도서관 유형                 954 non-null    int64  
 3   일평균                    954 non-null    int64  
 4   연간 총 운영일               954 non-null    int64  
 5   건물면적(㎡)                954 non-null    float64
 6   열람(석)                  954 non-null    int64  
 7   현재 총 보유장서의 수(권)        954 non-null    int64  
 8   현재 총 보유 연속간행물의 수(종)    954 non-null    int64  
 9   운영위원회
구성유무             954 non-null    object 
 10  정기모임 횟수                954 non-null    int64  
 11  PC현황 유무                954 non-null    object 
 12  상근직원(명)                954 non-null    int64  
 13  시간제직원(명)               954 non-null    int64  
 14  정기적자원봉사자(명)            954 non-null    int64  
 15  비정기적자원

### **데이터 전처리**

In [33]:
# 중복 컬럼인 주소 삭제
data.drop('주소', axis=1, inplace=True)
# 중복 행 지우기
data.drop_duplicates(inplace=True)

In [34]:
data.columns

Index(['도서관명', '분관여부', '도서관 유형', '일평균', '연간 총 운영일', '건물면적(㎡)', '열람(석)',
       '현재 총 보유장서의 수(권)', '현재 총 보유 연속간행물의 수(종)', '운영위원회\n구성유무', '정기모임 횟수',
       'PC현황 유무', '상근직원(명)', '시간제직원(명)', '정기적자원봉사자(명)', '비정기적자원봉사자(명)',
       '사서자격증보유자', '직원 교육시간(시간)', '재정자립도', '예산합계(원)', '도서관외대출여부', '연간도서대출권수',
       '연간 총 이용자수(명)', '등록된 \n총 이용자수\n(회원등록자 수)', '상호대차서비스 제공유무',
       '독서/문화프로그램 실시여부', '운영상태', '경도', '위도', '지번주소'],
      dtype='object')

In [35]:
# 컬럼명 변경
data.columns = ['Name','Branch', 'Type', 'Operating_hours', 'Operating_days', 'Area', 'Room', 'Books', 'Publications', 'Committee', 'Meeting', 'PC', 'Full_time', 'Part_time', 'Regular_volunteer', 'Irregular_volunteer',
                'License', 'Training_time', 'Financial', 'Budget', 'Off_library', 'Borrow', 'Target', 'Registration', 'Mutual_borrow', 'Program', 'Operation_status', 'Longitude', 'Latitude', 'Address']

#### 주소 데이터 이용하여 변수 추출

In [36]:
# 주소 데이터 접근, iterrows 사용가능
for index, row in data.iterrows():
  address = str(data.loc[index, 'Address'])
  data.loc[index, 'Gu'] = address.split(sep=' ')[1]

In [37]:
data[['Gu', 'Dong']]

,Gu,Dong
0,종로구,평동
1,종로구,평창동
2,종로구,무악동
3,종로구,숭인동
4,종로구,창신동
...,...,...
947,강동구,암사동
948,강동구,명일동
949,강동구,천호동
951,강동구,고덕동


#### 파생변수를 이용하여 병합

In [52]:
seoul_class = pd.read_excel('/content/drive/MyDrive/Contest/Library/Dataset/서울시 학급 수.xlsx')
seoul_population = pd.read_excel('/content/drive/MyDrive/Contest/Library/Dataset/서울시 구별 인구수.xlsx')

In [53]:
data_merge = data.merge(seoul_class, on='Gu', how='left')
data_merge = data_merge.merge(seoul_population, on='Gu', how='left')

In [54]:
data_merge.set_index

,Name,Branch,Type,Operating_hours,Operating_days,Area,Room,Books,Publications,Committee,...,0~9,10,20,30,40,50,60,70,80,90+
0,경희궁자이3단지 작은도서관,N,2,8,365,65.60,28,2556,0,N,...,7288,10941,27136,20882,22019,25935,20724,11712,6167,985
1,경희궁자이3단지 작은도서관,N,2,8,365,65.60,28,2556,0,N,...,7288,10941,27136,20882,22019,25935,20724,11712,6167,985
2,꿈꾸는 평창동 작은도서관,N,1,8,127,38.50,13,4654,7,Y,...,7288,10941,27136,20882,22019,25935,20724,11712,6167,985
3,꿈꾸는 평창동 작은도서관,N,1,8,127,38.50,13,4654,7,Y,...,7288,10941,27136,20882,22019,25935,20724,11712,6167,985
4,나무와 열매 어린이 도서관,N,2,8,200,132.00,25,5820,1,Y,...,7288,10941,27136,20882,22019,25935,20724,11712,6167,985
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
963,채우리작은도서관,N,1,7,240,100.00,30,12500,2,N,...,34507,38978,61129,72053,75518,72662,66853,31556,11298,1918
964,책꾸러미작은도서관,N,1,7,240,33.00,10,6811,0,Y,...,34507,38978,61129,72053,75518,72662,66853,31556,11298,1918
965,초대구름도서관,N,2,8,313,189.28,16,5368,9,Y,...,34507,38978,61129,72053,75518,72662,66853,31556,11298,1918
966,함께크는우리,N,2,5,199,132.00,80,6000,2,Y,...,34507,38978,61129,72053,75518,72662,66853,31556,11298,1918


### **전처리 2**